In [2]:
%load_ext autoreload
%autoreload 2

import rare_traffic_sign_solution as rtss
import lib

import torch
import torchvision
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from torchvision.datasets import ImageFolder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from math import floor, ceil
from sklearn.model_selection import train_test_split

import json
import shutil
import requests
import functools
import pathlib
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
import os
from collections import defaultdict

from IPython.display import clear_output

matplotlib.rcParams['figure.figsize'] = (20, 1)

MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

In [3]:
model = rtss.ModelWithHead()
model.load_nn('improved_features_model.pth')
model.load_head('knn_model.bin')

In [5]:
device = torch.device('cuda:0')
model.to(device)

In [4]:
dataset_test = rtss.TestData('smalltest', 'classes.json', 'smalltest_annotations.csv', return_class_name=False)
knn_test = DataLoader(dataset_test, batch_size=16, num_workers=16)

In [25]:
acc = 0
for batch in knn_test:
    images, _, labels = batch
    images.to(device)
    preds = model.predict(images)
    acc += (torch.Tensor(preds) == labels).type(torch.float).mean()
acc /= len(knn_test)
print(acc)

tensor(0.7394)


In [26]:
acc

tensor(0.7394)